# SageMaker Inference Recommender

This notebook has been tested in `Python 3 (TensorFlow 1.15 Python 3.7 CPU Optimized)` kernel on a `ml.g4dn.xlarge` instance

## Contents
[1. Introduction](#1.-Introduction)  
[2. Download the Model & payload](#2.-Download-the-Model-&-payload)  
[3. Machine Learning model details](#3.-Machine-Learning-model-details)  
[4. Register Model Version/Package](#4.-Register-Model-Version/Package)  
[5. Create a SageMaker Inference Recommender Default Job](#5:-Create-a-SageMaker-Inference-Recommender-Default-Job)   
[6. Instance Recommendation Results](#6.-Instance-Recommendation-Results)   
[7. Create a SageMaker Inference Recommender Advanced Job](#7.-Custom-Load-Test)  
[8. Describe result of an Advanced Job](#8.-Custom-Load-Test-Results)  

## 1. Introduction

SageMaker Inference Recommender is a new capability of SageMaker that reduces the time required to get machine learning (ML) models in production by automating performance benchmarking and load testing models across SageMaker ML instances. You can use Inference Recommender to deploy your model to a real-time inference endpoint that delivers the best performance at the lowest cost. 

Get started with Inference Recommender on SageMaker in minutes while selecting an instance and get an optimized endpoint configuration in hours, eliminating weeks of manual testing and tuning time.


To begin, let's install the wheels of the required packages ie SageMaker Python SDK, boto3, botocore and awscli

In [ ]:
!pip install sagemaker botocore boto3 awscli --upgrade

## 2. Download the Model & payload 

In this example, we are using Resnet50 Image Classification Model. Use Tensorflow 1.15 Python 3.7 kernel

In [ ]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
import time

region = boto3.Session().region_name
role = get_execution_role()
sagemaker_session = Session()

print(region)

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import backend

tf.keras.backend.set_learning_phase(0)
model = tf.keras.applications.ResNet50()

# Creating the directory strcture
model_version = "1"
export_dir = "./model/" + model_version
if not os.path.exists(export_dir):
    os.makedirs(export_dir)
    print("Directory ", export_dir, " Created ")
else:
    print("Directory ", export_dir, " already exists")

model_archive_name = "model.tar.gz"
payload_archive_name = "payload.tar.gz"

# Save to SavedModel
model.save(export_dir, save_format="tf", include_optimizer=False)

### Tar the model and code

In [ ]:
!tar -cvpzf model.tar.gz ./model ./code

### Download the payload 

In [ ]:
import sagemaker
from sagemaker import get_execution_role

SAMPLES_BUCKET = "sagemaker-sample-files"
PREFIX = "datasets/image/pets/"

payload_location = "./sample-payload/"

if not os.path.exists(payload_location):
    os.makedirs(payload_location)
    print("Directory ", payload_location, " Created ")
else:
    print("Directory ", payload_location, " already exists")

In [ ]:
!curl  https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/pets/boxer_dog.jpg > ./sample-payload/boxer_dog.jpg
!curl  https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/pets/british_blue_shorthair_cat.jpg > ./sample-payload/british_blue_shorthair_cat.jpg
!curl  https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/pets/english_cocker_spaniel_dog.jpg > ./sample-payload/english_cocker_spaniel_dog.jpg
!curl  https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/pets/shiba_inu_dog.jpg > ./sample-payload/shiba_inu_dog.jpg

### Tar the payload

In [ ]:
!cd ./sample-payload/ && tar czvf ../payload.tar.gz *

### Upload to S3

We now have a model archive ready. We need to upload it to S3 before we can use it with Inference Recommender. We'll use the SageMaker Python SDK to handle the upload.

We need to create an archive that contains individual files that Inference Recommender can send to your SageMaker Endpoints. Inference Recommender will randomly sample files from this archive so make sure it contains a similar distribution of payloads you'd expect in production. Note that your inference code must be able to read in the file formats from the sample payload.

In [ ]:
%%time

import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
import sagemaker
from sagemaker import get_execution_role

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
bucket = sagemaker.Session().default_bucket()

prefix = "sagemaker/inference-recommender"

sample_payload_url = sagemaker.Session().upload_data(
    payload_archive_name, bucket=bucket, key_prefix=prefix + "/inference"
)
model_url = sagemaker.Session().upload_data(
    model_archive_name, bucket=bucket, key_prefix=prefix + "/reset50/model"
)

print(sample_payload_url)
print(model_url)

## 3. Machine Learning model details

Inference Recommender uses information about your ML model to recommend the best instance types and endpoint configurations for deployment. You can provide as much or as little information as you'd like and Inference Recommender will use that to provide recommendations.

Example ML Domains: `COMPUTER_VISION`, `NATURAL_LANGUAGE_PROCESSING`, `MACHINE_LEARNING`

Example ML Tasks: `CLASSIFICATION`, `REGRESSION`, `OBJECT_DETECTION`, `OTHER`

Example Model name: `resnet50`, `yolov4`, `xgboost` etc

Use list_model_metadata API to fetch the list of available models. This will help you to pick the closest model for better recommendation. In this example, as we pick the `resnet50` model we selected `COMPUTER_VISION` as the Domain, `IMAGE_CLASSIFICATION` as the Task.

In [ ]:
import boto3
import pandas as pd

inference_client = boto3.client("sagemaker", region)

list_model_metadata_response = inference_client.list_model_metadata()

domains = []
frameworks = []
framework_versions = []
tasks = []
models = []

for model_summary in list_model_metadata_response["ModelMetadataSummaries"]:
    domains.append(model_summary["Domain"])
    tasks.append(model_summary["Task"])
    models.append(model_summary["Model"])
    frameworks.append(model_summary["Framework"])
    framework_versions.append(model_summary["FrameworkVersion"])

data = {
    "Domain": domains,
    "Task": tasks,
    "Framework": frameworks,
    "FrameworkVersion": framework_versions,
    "Model": models,
}

df = pd.DataFrame(data)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
pd.set_option("display.colheader_justify", "center")
pd.set_option("display.precision", 3)


display(df.sort_values(by=["Domain", "Task", "Framework", "FrameworkVersion"]))

### Container image URL

If you don’t have an inference container image, you can use one of the open source [deep learning containers (DLCs)](https://github.com/aws/deep-learning-containers) provided by AWS to serve your ML model.

In [ ]:
from sagemaker import image_uris

instance_type = "ml.c5.xlarge"  # Note: you can use any CPU-based instance type here, this is just to get CPU tagged image
# Adding framework related parameters
framework_name = "tensorflow"
framework_version = "1.15.4"

# ML model details
ml_domain = "COMPUTER_VISION"
ml_task = "IMAGE_CLASSIFICATION"
model_name = "resnet50"

dlc_uri = image_uris.retrieve(
    framework_name,
    region,
    version=framework_version,
    py_version="py3",
    instance_type=instance_type,
    image_scope="inference",
)
dlc_uri

## 4. Register Model Version/Package

Inference Recommender expects the model to be packaged in the model registry. Here, we are creating a model package group and a model package version. The model package version which takes container, model url etc, will now allow you to pass additional information about the model like `Domain`, `Task`, `Framework`, `FrameworkVersion`, `NearestModelName`, `SamplePayloadUrl`

As `SamplePayloadUrl` and `SupportedContentTypes` parameters are essential for benchmarking the endpoint. We also highly recommend you to specific `Domain`, `Task`, `Framework`, `FrameworkVersion`, `NearestModelName` for better inference recommendation. 

In [ ]:
import boto3
import uuid

inference_client = boto3.client("sagemaker", region)

model_package_group_name = uuid.uuid1()
print(model_package_group_name)
model_pacakge_group_response = inference_client.create_model_package_group(
    ModelPackageGroupName=str(model_package_group_name), ModelPackageGroupDescription="description"
)

print(model_pacakge_group_response)

model_package_version_response = inference_client.create_model_package(
    ModelPackageGroupName=str(model_package_group_name),
    ModelPackageDescription="InferenceRecommenderDemo",
    Domain=ml_domain,
    Task=ml_task,
    SamplePayloadUrl=sample_payload_url,
    InferenceSpecification={
        "Containers": [
            {
                "ContainerHostname": "dlc",
                "Image": dlc_uri,
                "ModelDataUrl": model_url,
                "Framework": "TENSORFLOW",
                "FrameworkVersion": framework_version,
                "NearestModelName": model_name,
                "ModelInput": {"DataInputConfig": '{"input_1":[1,3,224,224]}'},
            },
        ],
        "SupportedRealtimeInferenceInstanceTypes": [
            "ml.c5.xlarge",
            "ml.c5.2xlarge",
            "ml.m5.xlarge",
            "ml.m5.2xlarge",
            "ml.m5.4xlarge",
            "ml.r5.large",
            "ml.r5.xlarge",
            "ml.r5.2xlarge",
            "ml.r5.4xlarge",
            "ml.r5.12xlarge",
            "ml.r5.24xlarge",
            "ml.r5d.large",
            "ml.r5d.xlarge",
            "ml.r5d.2xlarge",
            "ml.r5d.4xlarge",
            "ml.r5d.12xlarge",
            "ml.r5d.24xlarge",
            "ml.inf1.xlarge",
            "ml.inf1.2xlarge",
            "ml.inf1.6xlarge",
            "ml.inf1.24xlarge",
        ],
        "SupportedContentTypes": [
            "application/x-image",
        ],
        "SupportedResponseMIMETypes": [],
    },
)

print(model_package_version_response)

## 5: Create a SageMaker Inference Recommender Default Job

Now with your model in Model Registry, you can kick off a 'Default' job to get instance recommendations. This only requires your `ModelPackageVersionArn` and comes back with recommendations within an hour. 

The output is a list of instance type recommendations with associated environment variables, cost, throughput and latency metrics.

In [ ]:
import boto3
import uuid
from sagemaker import get_execution_role

client = boto3.client("sagemaker", region)

role = get_execution_role()
default_job = uuid.uuid1()
default_response = client.create_inference_recommendations_job(
    JobName=str(default_job),
    JobDescription="Job Description",
    JobType="Default",
    RoleArn=role,
    InputConfig={"ModelPackageVersionArn": model_package_version_response["ModelPackageArn"]},
)

print(default_response)

### 6. Instance Recommendation Results

Inference recommender job provides multiple endpoint recommendations in its result. The recommendation includes `InstanceType`, `InitialInstanceCount`, `EnvironmentParameters` which includes tuned parameters for better performance. We also include the benchmarking results like `MaxInvocations`, `ModelLatency`, `CostPerHour` and `CostPerInference` for deeper analysis. We believe the information provided  will help you narrow down to a specific endpoint configuration that suits your use case. 

Example:   

If your motivation is overall price-performance, then you should focus on `CostPerInference` metrics  
If your motivation is latency/throughput, then you should focus on `ModelLatency` / `MaxInvocations` metrics

In [ ]:
import boto3
import uuid
import pprint
import pandas as pd

client = boto3.client("sagemaker", region)

stopped = False
while not stopped:
    inference_recommender_job = client.describe_inference_recommendations_job(
        JobName=str(default_job)
    )
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        stopped = True
    else:
        print("Infernece recommender job in progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Infernece recommender job failed ")
    print("Failed Reason: {}".inference_recommender_job["FailedReason"])
else:
    print("Infernece recommender job completed")

### Detailing out the result

In [ ]:
data = [
    {**x["EndpointConfiguration"], **x["ModelConfiguration"], **x["Metrics"]}
    for x in inference_recommender_job["InferenceRecommendations"]
]
df = pd.DataFrame(data)
df.drop("VariantName", inplace=True, axis=1)
pd.set_option("max_colwidth", 400)
df.head()

## 7. Custom Load Test

With an 'Advanced' job, you can provide your production requirements, select instance types, tune environment variables and perform more extensive load tests. This typically takes 2 hours depending on your traffic pattern and number of instance types. 

The output is a list of endpoint configuration recommendations (instance type, instance count, environment variables) with associated cost, throughput and latency metrics.

In the below example, we are tuning the endpoint against an environment variable `OMP_NUM_THREADS` with two values `[2, 4]` and we aim to limit the  latency requirement to `100` ms. The goal is to find which endpoint configuration provides the best performance. 

In [ ]:
import boto3
import uuid

client = boto3.client("sagemaker", region)

role = get_execution_role()
advanced_job = uuid.uuid1()
advanced_response = client.create_inference_recommendations_job(
    JobName=str(advanced_job),
    JobDescription="JobDescription",
    JobType="Advanced",
    RoleArn=role,
    InputConfig={
        "ModelPackageVersionArn": model_package_version_response["ModelPackageArn"],
        "JobDurationInSeconds": 7200,
        "EndpointConfigurations": [
            {
                "InstanceType": "ml.m5.xlarge",
                "EnvironmentParameterRanges": {
                    "CategoricalParameterRanges": [{"Name": "OMP_NUM_THREADS", "Value": ["2", "4"]}]
                },
            }
        ],
        "ResourceLimit": {"MaxNumberOfTests": 2, "MaxParallelOfTests": 1},
        "TrafficPattern": {
            "TrafficType": "PHASES",
            "Phases": [{"InitialNumberOfUsers": 1, "SpawnRate": 1, "DurationInSeconds": 120}],
        },
    },
    StoppingConditions={
        "MaxInvocations": 300,
        "ModelLatencyThresholds": [{"Percentile": "P95", "ValueInMilliseconds": 100}],
    },
)

print(advanced_response)

### 8. Custom Load Test Results

Inference recommender does benchmarking on both the endpoint configurations and here is the result. 

Analyzing load test result,    
`OMP_NUM_THREADS` = 2 shows ~20% better throughput when compared to `OMP_NUM_THREADS` = 4   
`OMP_NUM_THREADS` = 2 shows 25% saving in inference-cost when compared to `OMP_NUM_THREADS` = 4   

In all front, `OMP_NUM_THREADS` = 2  is much better endpoint configuration than `OMP_NUM_THREADS` = 4 

In [ ]:
import boto3
import uuid
import pprint
import pandas as pd

client = boto3.client("sagemaker", region)

stopped = False
while not stopped:
    inference_recommender_job = client.describe_inference_recommendations_job(
        JobName=str(advanced_job)
    )
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        stopped = True
    else:
        print("Infernece recommender job in progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Infernece recommender job failed ")
    print("Failed Reason: {}".inference_recommender_job["FailedReason"])
else:
    print("Infernece recommender job completed")

### Detailing out the result

In [ ]:
data = [
    {**x["EndpointConfiguration"], **x["ModelConfiguration"], **x["Metrics"]}
    for x in inference_recommender_job["InferenceRecommendations"]
]
df = pd.DataFrame(data)
df.drop("VariantName", inplace=True, axis=1)
pd.set_option("max_colwidth", 400)
df.head()